# DAT257x: Reinforcement Learning Explained

## Lab 4: Dynamic Programming

### Exercise 4.3 Policy Iteration

Policy Iteration calculates the optimal policy for an MDP, given its full definition.  The full definition of an MDP is the set of states, the set of available actions for each state, the set of rewards, the discount factor, and the state/reward transition function.

In [1]:
import test_dp               # required for testing and grading your code
import gridworld_mdp as gw   # defines the MDP for a 4x4 gridworld
import numpy as np

**Implement the algorithm for Policy Iteration**.  Policy Iteration calculates the optimal policy for an MDP by doing repeated steps of policy evaluation and policy improvement.

A empty function **policy_iteration** is provided below; implement the body of the function to correctly calculate the optimal policy for an MDP.  The function defines 5 parameters - a definition of each parameter is given in the comment block for the function.  For sample parameter values, see the calling code in the cell following the function.

Note that there is a subtle difference between the algorithm for Policy Evaluation, which assumes the policy is stochastic, and the Policy Evaluation step for the Policy Iteration algorithm, which assumes the policy is deterministic.  This means that you cannot directly call your previous code, but you can reuse large pieces of it for the Policy Evaluation step.

In [2]:
def policy_iteration(state_count, gamma, theta, get_available_actions, get_transitions):
    """
    This function computes the optimal value function and policy for the specified MDP, using the Policy Iteration algorithm.
    'state_count' is the total number of states in the MDP. States are represented as 0-relative numbers.
    
    'gamma' is the MDP discount factor for rewards.
    
    'theta' is the small number threshold to signal convergence of the value function (see Iterative Policy Evaluation algorithm).
    
    'get_available_actions' returns a list of the MDP available actions for the specified state parameter.
    
    'get_transitions' is the MDP state / reward transiton function.  It accepts two parameters, state and action, and returns
        a list of tuples, where each tuple is of the form: (next_state, reward, probabiliity).  
    """
    V = state_count*[0]              # init all state value estimates to 0
    pi = state_count * [0]
    
    # init with a policy with first avail action for each state
    for s in range(state_count):
        avail_actions = get_available_actions(s)
        pi[s] = avail_actions[0]
     
    #Evaluates the value of a state given the current action from the state
    def evaluate_policy(values, actions):
        while True:        
            delta = 0
            for state in range(state_count):                
                next_state, reward, probability = get_transitions(state, actions[state])[0]
                v = probability * (reward + gamma * values[next_state])
                delta = max(delta, np.abs(values[state] - v))
                V[state] = v        
            
            if delta < theta:
                break
         
        return V

    while True:        

        #Policy Evaluation
        V = evaluate_policy(V, pi)
        
        
        #Policy improvement   
        #Once we have a policy, we choose a policy from the available actions that is the best move from the 
        #current state, if thats different from the existing actions, we will need to evaluate the policy
        #again with the new set of actions from the state. We stop when two subsequent improvements gives us the 
        #same action
        
        policy_stable = True    
        for state in range(state_count):            
            available_actions = [act for act in get_available_actions(state)] 
            #For each state and action, get possible next state's values. We use these next states to find max value
            neighbor_values = [V[get_transitions(state, a)[0][0]] for a in available_actions]
            greedy_transition = np.argmax(neighbor_values)

            if pi[state] != avail_actions[greedy_transition]:
                policy_stable = False
                pi[state] = avail_actions[greedy_transition]

                
        if policy_stable:
            break
        
    return (V, pi)        # return both the final value function and the final policy

First, test our function using the MDP defined by gw.* functions.

In [3]:
n_states = gw.get_state_count()

# test our function
values, policy = policy_iteration(state_count=n_states, gamma=.9, theta=.001, get_available_actions=gw.get_available_actions, \
    get_transitions=gw.get_transitions)

print("Values=", values)
print("Policy=", policy)

Values= [0.0, -1.0, -1.9, -2.71, -1.0, -1.9, -2.71, -1.9, -1.9, -2.71, -1.9, -1.0, -2.71, -1.9, -1.0]
Policy= ['up', 'left', 'left', 'down', 'up', 'up', 'up', 'down', 'up', 'up', 'down', 'down', 'up', 'right', 'right']


**Expected output from running above cell:**

`
Values= [0.0, -1.0, -1.9, -2.71, -1.0, -1.9, -2.71, -1.9, -1.9, -2.71, -1.9, -1.0, -2.71, -1.9, -1.0]
Policy= ['up', 'left', 'left', 'down', 'up', 'up', 'up', 'down', 'up', 'up', 'down', 'down', 'up', 'right', 'right']
`

In [4]:
import numpy as np
a = np.append(values, 0)
np.reshape(a, (4,4))

array([[ 0.  , -1.  , -1.9 , -2.71],
       [-1.  , -1.9 , -2.71, -1.9 ],
       [-1.9 , -2.71, -1.9 , -1.  ],
       [-2.71, -1.9 , -1.  ,  0.  ]])

Now, test our function using the test_dp helper.  The helper also uses the gw MDP, but with a different gamma value.
If our function passes all tests, a passcode will be printed.

In [5]:
a = np.append(policy, policy[0])
np.reshape(a, (4,4))

array([['up', 'left', 'left', 'down'],
       ['up', 'up', 'up', 'down'],
       ['up', 'up', 'down', 'down'],
       ['up', 'right', 'right', 'up']], 
      dtype='<U5')

In [6]:
# test our function using the test_db helper
test_dp.policy_iteration_test( policy_iteration ) 


Testing: Policy Iteration
passed test: return value is tuple
passed test: length of tuple = 2
passed test: v is list of length=15
passed test: values of v elements
passed test: pi is list of length=15
passed test: values of pi elements
PASSED: Policy Iteration passcode = 9970-010
